# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher_Linux_v2/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

T = 8
episodes = 100000
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
import torch
from ppo_agent20 import Agent
from collections import deque
           # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

In [6]:
BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 512        # minibatch size
GAMMA = 0.9            # discount factor
LR_ACTOR = 5e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay
random_seed=10



In [10]:
import torch
episodes = 1200
from collections import namedtuple

# agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)
# agent.actor_local.load_state_dict(torch.load('actor_net.chpt'))
# agent.critic_local.load_state_dict(torch.load('critic_net.chpt'))

reward_list = []

agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)
scores_mean = deque(maxlen=1000)
scores = []
best_score = 0
best_average_score = 0
episodes=2000
max_steps=1000
Transition = namedtuple('Transition', ['state', 'action',  'a_log_prob', 'reward', 'next_state'])



for episode in range(1, episodes+1):

    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    cur_states = env_info.vector_observations  
    scores_agents = np.zeros(num_agents)      
    
    
    for step in range(max_steps):
        
        #should be added prbabilities to agent.step
        actions, probs = agent.act(cur_states)
        env_info = env.step(actions)[brain_name]     
        next_states = env_info.vector_observations   
        rewards = env_info.rewards                   
        dones = env_info.local_done                  
      
        trans = Transition(cur_states, actions, probs,  rewards, next_states)

        #should be added storing transitions
        #agent.step(, dones, step)
        
        agent.store_transition(trans)
        
        cur_states = next_states
        scores_agents += rewards
        if step>900:
            agent.learn(episode)
            break
    
    score = np.mean(scores_agents)
    scores_mean.append(score)
    average_score = np.mean(scores_mean)
    scores.append(score)
    if score > best_score:
        best_score = score
    if average_score > best_average_score:
        best_average_score = average_score
    print("Episode:{}, Average Score:{:.2f}".format(episode, average_score))
    if average_score > 30:
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        print("Average score of 30 achieved")            
        break
    
env.close()

PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1147)
PPO step 5
PPO step 6
PPO step 7
Episode:1, Average Score:0.51
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1016)
val_loss tensor(1.00000e-02 *
       9.5164)
PPO step 5
PPO step 6
PPO step 7
Episode:2, Average Score:0.38
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1031)
val_loss tensor(1.00000e-02 *
       8.8149)
val_loss tensor(1.00000e-02 *
       9.3679)
PPO step 5
PPO step 6
PPO step 7
Episode:3, Average Score:0.34
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       9.6821)
val_loss tensor(1.00000e-02 *
       9.2915)
val_loss tensor(1.00000e-02 *
       9.6842)
val_loss tensor(1.00000e-02 *
       8.9886)
PPO step 5
PPO step 6
PPO step 7
Episode:4, Average Score:0.34
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.9637)
val_loss tensor(1.00000e-02 *
       9.41

PPO step 6
PPO step 7
Episode:30, Average Score:0.85
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5876)
PPO step 5
PPO step 6
PPO step 7
Episode:31, Average Score:0.87
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5057)
val_loss tensor(0.5113)
PPO step 5
PPO step 6
PPO step 7
Episode:32, Average Score:0.89
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5595)
val_loss tensor(0.5144)
val_loss tensor(0.5195)
PPO step 5
PPO step 6
PPO step 7
Episode:33, Average Score:0.90
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5203)
val_loss tensor(0.5195)
val_loss tensor(0.5212)
val_loss tensor(0.5176)
PPO step 5
PPO step 6
PPO step 7
Episode:34, Average Score:0.92
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.4998)
val_loss tensor(0.5129)
val_loss tensor(0.5077)
val_loss tensor(0.4919)
val_loss tensor(0.5153)
PPO step 5
PPO step 6
PPO step 7
Episode:35, Average Sco

PPO step 4
val_loss tensor(0.5951)
val_loss tensor(0.5906)
val_loss tensor(0.5872)
val_loss tensor(0.6100)
val_loss tensor(0.6136)
PPO step 5
PPO step 6
PPO step 7
Episode:65, Average Score:1.07
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5893)
val_loss tensor(0.5838)
val_loss tensor(0.5870)
val_loss tensor(0.5712)
val_loss tensor(0.5820)
val_loss tensor(0.5987)
PPO step 5
PPO step 6
PPO step 7
Episode:66, Average Score:1.07
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5845)
val_loss tensor(0.5626)
val_loss tensor(0.5775)
val_loss tensor(0.5802)
val_loss tensor(0.5770)
val_loss tensor(0.5889)
val_loss tensor(0.5896)
PPO step 5
PPO step 6
PPO step 7
Episode:67, Average Score:1.07
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.5780)
val_loss tensor(0.5898)
val_loss tensor(0.5843)
val_loss tensor(0.5811)
val_loss tensor(0.5809)
val_loss tensor(0.5853)
val_loss tensor(0.5595)
val_loss tensor(0.5975)
PPO step

val_loss tensor(0.1515)
val_loss tensor(0.1655)
val_loss tensor(0.1514)
val_loss tensor(0.1717)
val_loss tensor(0.1696)
val_loss tensor(0.1740)
val_loss tensor(0.2219)
PPO step 5
PPO step 6
PPO step 7
Episode:98, Average Score:0.89
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1711)
val_loss tensor(0.1401)
val_loss tensor(0.1606)
val_loss tensor(0.1551)
val_loss tensor(0.1542)
val_loss tensor(0.1568)
val_loss tensor(0.1548)
val_loss tensor(0.1573)
PPO step 5
PPO step 6
PPO step 7
Episode:99, Average Score:0.88
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1656)
val_loss tensor(0.1431)
val_loss tensor(0.1673)
val_loss tensor(0.1480)
val_loss tensor(0.1619)
val_loss tensor(0.1443)
val_loss tensor(0.1538)
val_loss tensor(0.1421)
val_loss tensor(0.1511)
PPO step 5
PPO step 6
PPO step 7
Episode:100, Average Score:0.87
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1611)
PPO step 5
PPO step 6
PPO step 7
Episode:1

val_loss tensor(0.1058)
val_loss tensor(1.00000e-02 *
       9.3317)
val_loss tensor(1.00000e-02 *
       9.3494)
val_loss tensor(0.1004)
val_loss tensor(0.1012)
PPO step 5
PPO step 6
PPO step 7
Episode:129, Average Score:0.70
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       9.9804)
val_loss tensor(1.00000e-02 *
       8.8136)
val_loss tensor(1.00000e-02 *
       8.6675)
val_loss tensor(1.00000e-02 *
       9.9400)
val_loss tensor(1.00000e-02 *
       8.3691)
val_loss tensor(1.00000e-02 *
       8.9739)
val_loss tensor(1.00000e-02 *
       9.7058)
val_loss tensor(1.00000e-02 *
       9.0410)
val_loss tensor(0.1030)
PPO step 5
PPO step 6
PPO step 7
Episode:130, Average Score:0.69
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(0.1102)
PPO step 5
PPO step 6
PPO step 7
Episode:131, Average Score:0.69
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.5032)
val_loss tensor(1.00000e-

...

val_loss tensor(1.00000e-02 *
       4.7495)
val_loss tensor(1.00000e-02 *
       4.3141)
val_loss tensor(1.00000e-02 *
       5.7756)
val_loss tensor(1.00000e-02 *
       5.2965)
val_loss tensor(1.00000e-02 *
       5.6850)
val_loss tensor(1.00000e-02 *
       6.6659)
val_loss tensor(1.00000e-02 *
       5.1517)
PPO step 5
PPO step 6
PPO step 7
Episode:1889, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       6.6785)
val_loss tensor(1.00000e-02 *
       5.7805)
val_loss tensor(1.00000e-02 *
       8.1178)
val_loss tensor(1.00000e-02 *
       5.4046)
val_loss tensor(1.00000e-02 *
       6.7556)
val_loss tensor(1.00000e-02 *
       7.2169)
val_loss tensor(1.00000e-02 *
       5.8145)
val_loss tensor(1.00000e-02 *
       8.3481)
val_loss tensor(1.00000e-02 *
       6.8924)
PPO step 5
PPO step 6
PPO step 7
Episode:1890, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7

PPO step 4
val_loss tensor(1.00000e-02 *
       7.8353)
val_loss tensor(1.00000e-02 *
       5.6435)
PPO step 5
PPO step 6
PPO step 7
Episode:1912, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.2751)
val_loss tensor(1.00000e-02 *
       6.1511)
val_loss tensor(1.00000e-02 *
       6.9149)
PPO step 5
PPO step 6
PPO step 7
Episode:1913, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.1486)
val_loss tensor(1.00000e-02 *
       7.9426)
val_loss tensor(1.00000e-02 *
       7.7991)
val_loss tensor(1.00000e-02 *
       7.4587)
PPO step 5
PPO step 6
PPO step 7
Episode:1914, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.5788)
val_loss tensor(1.00000e-02 *
       8.5433)
val_loss tensor(1.00000e-02 *
       8.8384)
val_loss tensor(1.00000e-02 *
       8.6962)
val_loss tensor(0.1185)
PPO step 5
PPO st

val_loss tensor(1.00000e-02 *
       7.7362)
val_loss tensor(1.00000e-02 *
       8.4344)
val_loss tensor(1.00000e-02 *
       7.3559)
val_loss tensor(1.00000e-02 *
       7.4221)
val_loss tensor(1.00000e-02 *
       8.9256)
PPO step 5
PPO step 6
PPO step 7
Episode:1936, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.5589)
val_loss tensor(1.00000e-02 *
       7.8201)
val_loss tensor(1.00000e-02 *
       9.3900)
val_loss tensor(1.00000e-02 *
       8.4940)
val_loss tensor(1.00000e-02 *
       9.5669)
val_loss tensor(1.00000e-02 *
       9.4166)
val_loss tensor(1.00000e-02 *
       7.5116)
PPO step 5
PPO step 6
PPO step 7
Episode:1937, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       8.6732)
val_loss tensor(0.1141)
val_loss tensor(1.00000e-02 *
       8.9955)
val_loss tensor(0.1020)
val_loss tensor(0.1083)
val_loss tensor(1.00000e-02 *
       8.7302)
val_loss te

val_loss tensor(1.00000e-02 *
       7.6457)
val_loss tensor(1.00000e-02 *
       6.2413)
val_loss tensor(1.00000e-02 *
       8.3016)
val_loss tensor(1.00000e-02 *
       8.6525)
val_loss tensor(1.00000e-02 *
       6.0187)
PPO step 5
PPO step 6
PPO step 7
Episode:1959, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.3664)
val_loss tensor(1.00000e-02 *
       9.8282)
val_loss tensor(1.00000e-02 *
       7.7847)
val_loss tensor(1.00000e-02 *
       6.3839)
val_loss tensor(1.00000e-02 *
       7.1529)
val_loss tensor(1.00000e-02 *
       8.4752)
val_loss tensor(1.00000e-02 *
       6.9062)
val_loss tensor(1.00000e-02 *
       8.1240)
val_loss tensor(1.00000e-02 *
       4.7242)
PPO step 5
PPO step 6
PPO step 7
Episode:1960, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       4.4973)
PPO step 5
PPO step 6
PPO step 7
Episode:1961, Average Score:0.10
PPO step 0
PPO st

PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.5846)
val_loss tensor(1.00000e-02 *
       7.8330)
PPO step 5
PPO step 6
PPO step 7
Episode:1982, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.5477)
val_loss tensor(1.00000e-02 *
       7.2352)
val_loss tensor(1.00000e-02 *
       6.0155)
PPO step 5
PPO step 6
PPO step 7
Episode:1983, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.4732)
val_loss tensor(1.00000e-02 *
       5.3188)
val_loss tensor(1.00000e-02 *
       6.4096)
val_loss tensor(1.00000e-02 *
       5.5830)
PPO step 5
PPO step 6
PPO step 7
Episode:1984, Average Score:0.10
PPO step 0
PPO step 1
PPO step 2
PPO step 3
PPO step 4
val_loss tensor(1.00000e-02 *
       7.2884)
val_loss tensor(1.00000e-02 *
       4.7546)
val_loss tensor(1.00000e-02 *
       5.0767)
val_loss tensor(1.00000e-02 *
       6.3561)
val_loss tensor(1.0

In [15]:
import torch
from collections import deque
from time import sleep

num_agents=20
agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)

agent.actor_local.load_state_dict(torch.load('checkpoint_actor_ppo449.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic_ppo449.pth'))


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
cur_states = env_info.vector_observations  
scores_agents = np.zeros(num_agents)

for step in range(1100):
    actions = agent.act(cur_states)[0]
    env_info = env.step(actions)[brain_name]     
    next_states = env_info.vector_observations   
    rewards = env_info.rewards                   
    dones = env_info.local_done                  
    cur_states = next_states
    scores_agents += rewards
    if np.any(dones):
        break

            
env.close()

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```